## Cargando datos .csv

Esta guía provee un ejemplo de como cargar datos .csv desde un archivo dentro de **tf.data.Dataset** 

Los datos utilizados en este tutorial son tomados de **Titanic Passenger List**. El modelo busca predecir que tan 
probable es que un pasajero sobreviva basado en sus características como la edad, genero, clase y si la persona 
está viajando sola.

El primer paso es importar los módulos necesarios y descargar los archivos .csv

In [1]:
import functools
import tensorflow as tf
import numpy as np

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 1us/step


In [3]:
# Hacer los valores de NumPy más fáciles de leer.
np.set_printoptions(precision=3, suppress=True)

### Carga los datos

Para empezar, vamos a ver el encabezado de el archivo csv para observar cómo está organizado. Para esto usaremos la 
librería **pandas** que nos despliega la información en forma de tablas.

In [4]:
import pandas as pd

data = pd.read_csv(train_file_path)

In [5]:
data.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


La columna mas importante que se tiene que identificar es la que contiene los valores que estamos intentado predecir, 
la columna **survived** que tiene valores binarios.

In [6]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

Ahora lee el archivo csv desde el archivo y crea un dataset.

In [7]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [8]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

Cada item en el dataset es un batch (o lote), representado como una tupla de (ejemplos, etiquetas). Los datos del 
ejemplo son organizados en columnas basadas en tensores (preferentemente que filas basadas en tensores), organizados 
lotes con un tamaño especifico. 

Probablemente sea mas comprensible si lo ves por ti mismo.

In [9]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
age                 : [28. 22. 23. 33. 28.]
n_siblings_spouses  : [0 0 0 1 0]
parch               : [0 0 0 0 0]
fare                : [ 8.05   9.35   7.896 53.1   56.496]
class               : [b'Third' b'Third' b'Third' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'E' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'y']


Como se puede ver, las columnas cuentan con nombres. El dataset constructor va a tomar estos nombres de manera 
automática. Si el archivo con el que estas trabajando no contiene los nombres en la primera línea puedes pasarlos como 
lista de String al argumento **columna_names** en la función **make_csv_dataset**

In [10]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'female' b'male' b'male' b'female' b'male']
age                 : [28. 60. 28. 45. 28.]
n_siblings_spouses  : [0 0 0 0 1]
parch               : [0 0 0 0 0]
fare                : [ 7.879 26.55  56.496  7.75  15.5  ]
class               : [b'Third' b'First' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'y' b'y' b'y' b'y' b'n']


En este ejemplo usaremos todas las columnas disponibles, si tu ocupas omitir alguna de las columnas del dataset, 
crea una lista con las columnas que planeas usar, y lo puedes pasarlo al argumento **select_columns** del constructor.

In [11]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [19. 27. 31. 32. 22.]
n_siblings_spouses  : [1 0 0 0 1]
class               : [b'Third' b'Second' b'First' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'A' b'unknown' b'unknown']
alone               : [b'n' b'y' b'y' b'y' b'n']


### Data preprocessing

Un archivo csv puede contener variedad de tipos de datos. Usualmente ocupamos convertir esa variedad de datos a un tipo
 en común con el cual se pueda describir de manera adecuada la información.

Puedes pre procesar la información con cualquier herramienta que desees (como nltk o sklearn), y solo para la 
información resultante a TensorFlow.

La principal ventaja de el pre procesamiento de información, es que cuando exportas el modelo este incluye el pre 
procesamiento. De esta manera tu puedes usar la información pre procesada dentro de cualquier modelo que desees.

#### Datos continuos
Si tu información ya esta contenida en el formato numérico correcto, puedes agrupar los datos en un vector antes de que 
lo pases a tu modelo.

In [12]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [ 2. 34. 51. 28.  4.]
n_siblings_spouses  : [4. 0. 0. 1. 1.]
parch               : [1. 0. 0. 0. 1.]
fare                : [29.125 13.    26.55  24.    16.7  ]


In [13]:
example_batch, labels_batch = next(iter(temp_dataset))

Aquí hay una función simple que empaquetara todas las columnas

In [14]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

Aplica esto a cada elemento en el dataset

In [15]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[31.     0.     0.    13.   ]
 [33.     0.     0.    12.275]
 [39.     1.     1.    79.65 ]
 [18.     0.     0.    13.   ]
 [38.     1.     0.    71.283]]

[1 0 1 0 1]


Si tienes múltiples tipos de datos, probablemente la mejor idea es que se separaran en campos numéricos. 
**tf.feature_column** api puede ayudarte a manejar este proceso.

In [16]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
age                 : [19. 52. 45. 39. 47.]
n_siblings_spouses  : [0 0 0 1 0]
parch               : [0 0 0 1 0]
fare                : [10.5   30.5   26.55  79.65  34.021]
class               : [b'Second' b'First' b'First' b'First' b'First']
deck                : [b'unknown' b'C' b'unknown' b'E' b'D']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'y']


In [17]:
example_batch, labels_batch = next(iter(temp_dataset)) 

Podemos definir una función para pre procesar la información mas general que empaquete una lista de números que 
representan una característica en especifico dentro de una sola columna.

In [18]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [19]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [20]:
show_batch(packed_train_data)

sex                 : [b'female' b'male' b'male' b'female' b'female']
class               : [b'First' b'First' b'Third' b'First' b'Third']
deck                : [b'B' b'B' b'unknown' b'D' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Cherbourg']
alone               : [b'n' b'y' b'n' b'n' b'y']
numeric             : [[ 15.      0.      1.    211.337]
 [ 40.      0.      0.      0.   ]
 [ 14.      5.      2.     46.9  ]
 [ 60.      1.      0.     75.25 ]
 [ 15.      0.      0.      7.225]]


In [21]:
example_batch, labels_batch = next(iter(packed_train_data)) 

#### Normalización de datos
Los datos siempre tienes que estar normalizados para que entes dentro del mismo rango y la información pueda ser 
analizada correctamente.

In [22]:
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [23]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [24]:
def normalize_numeric_data(data, mean, std):
  return (data-mean)/std

Ahora pueces crear una columna numérica, La API **tf.feature_columns.numeric_column** acepta como argumento 
normalizer_fn, con el cual puedes correr cada lote.

In [25]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000002BA62F5FAF8>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [26]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[22.   ,  0.   ,  0.   ,  9.35 ],
       [17.   ,  0.   ,  0.   ,  8.663],
       [34.   ,  1.   ,  0.   , 21.   ],
       [28.   ,  0.   ,  0.   ,  8.05 ],
       [48.   ,  1.   ,  0.   , 39.6  ]], dtype=float32)>

In [27]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.61 , -0.474, -0.479, -0.459],
       [-1.01 , -0.474, -0.479, -0.471],
       [ 0.349,  0.395, -0.479, -0.245],
       [-0.13 , -0.474, -0.479, -0.482],
       [ 1.468,  0.395, -0.479,  0.096]], dtype=float32)

La normalización basada en la media utilizada aquí requiere conocer de antemano las medias de cada columna.

Utiliza la API **tf.feature_column** para crear una colección con **tf.feature_column.indicator_column** para 
cada columna categórica.

In [28]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [29]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [30]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [31]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


Esta información se convertirá en parte de los datos de entrada una vez estemos construyendo el modelo.

Agrega las dos columnas de características y utilízalas como argumento para **tf.keras.layers.DenseFeatures** para crear 
una capa de entrada que extraiga y procese ambas columnas.

In [33]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.61  -0.474
 -0.479 -0.459  1.     0.   ]


### Construyendo el modelo
Construye un modelo **tf.keras.Sequential**, espesando con el preprocessing_layer (capa de procesamiento).

In [34]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

#### Entrena, evalúa y predice

Ahora el modelo puede ser instanciado y entrenado

In [35]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 0s 3ms/step - loss: 0.5129 - accuracy: 0.7352
Epoch 2/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4207 - accuracy: 0.8150
Epoch 3/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4037 - accuracy: 0.8278
Epoch 4/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3901 - accuracy: 0.8421
Epoch 5/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3918 - accuracy: 0.8325
Epoch 6/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3748 - accuracy: 0.8325
Epoch 7/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3711 - accuracy: 0.8389
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3638 - accuracy: 0.8437
Epoch 9/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3545 - accurac

Una vez el modelo sea entrenado, puedes medir la exactitud en base al **test_data**

In [36]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 1ms/step - loss: 0.4500 - accuracy: 0.8333


Test Loss 0.45002099871635437, Test Accuracy 0.8333333134651184


In [37]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Consider rewriting this model with the Functional API.
Predicted survival: 4.41%  | Actual outcome:  DIED
Predicted survival: 2.22%  | Actual outcome:  SURVIVED
Predicted survival: 68.67%  | Actual outcome:  DIED
Predicted survival: 99.90%  | Actual outcome:  DIED
Predicted survival: 6.47%  | Actual outcome:  SURVIVED
